## Use keras model to generate graffiti detections
This example demonstrates how to use an **exported keras checkpoint** to generate detections for a set of images.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
from utils import data_sequence, classification_auxiliary
import generateAnnotationFromModelsPredictions, eval_model
from utils import losses

Load the model from an exported checkpoint file that is placed inside the model_dir:

In [ ]:
model_dir = 'models/keras_models/Lfm_SeparatedParams_drop_0.5/'
weight_ckpt = 'combined_loss_alpha=2.50__weights_final__epoch=140__val_loss=21.02.hdf5'
model = eval_model.generate_model(model_dir,os.path.join(model_dir,weight_ckpt))

It is important to pass **keras_preprocessing_mode=True** parameter to the **Prediction_Data_Sequence** call when using keras models.
In addition the data sequence requires image_size, grid_sizes 
The path_to_test_images contains the images. It is recommended that this directory should contain images only.
The Prediction_Data_Sequence produces batches of numpy image arrays and file paths where the predictions can be stored.
The **min_scale** and **max_scale** parameters should math the values in the model's pipeline.yaml (located in the model_dir).
To ensure that the default anchor boxes are assigend properly **gride_sizes** should also fit the models feature pyramid shape.

In [ ]:
path_to_test_images = 'data/images/detection_test'
data_limit_of_n_images = 10
image_size = model.input_shape[1:]
grid_sizes = []
for output_tensor in model.outputs:
    grid_sizes.append(output_tensor.shape[1].value)

In [ ]:
data_generator = data_sequence.Prediction_Data_Sequence(
    path_to_test_images,
    keras_preprocessing_mode=True,
    max_data_size=data_limit_of_n_images,
    image_size=image_size,
    grid_sizes=grid_sizes,
    batch_size=4,
    min_scale=0.1,
    max_scale=0.9,
)

Use detection graph to run inference for each batch. The final detections are generated from the predictions according to the specified parameters.

In [ ]:
maximum_overlap_between_distinct_detections = 0.4
only_consider_objects_with_higher_confidence_than = 0.2
detect_at_most_n_distinct_objects = 10

In [ ]:
for h in range(len(data_generator)):
    x_batch,y_batch = data_generator[h]
    batch_predictions = eval_model.generate_keras_prediction_batch(x_batch,model,data_generator)
    for i in range(batch_predictions.shape[0]):
        predictions = batch_predictions[i]
        positives,_ = eval_model.detect(
            predictions,
            iou_overlap_threshold=maximum_overlap_between_distinct_detections,
            confidence_threshold=only_consider_objects_with_higher_confidence_than,
            max_number_of_detections=detect_at_most_n_distinct_objects,
        )
        pred_image_np = classification_auxiliary.invert_tf_preprocessor(x_batch[i])
        eval_model.plot_to_img(pred_image_np,positives,predictions,'red',12,True)
        eval_model.plot_to_img(pred_image_np,positives,predictions,'white',6,True)
        plt.imshow(pred_image_np)
        plt.show()